In [ ]:
%pip install gurobipy>=10
import gurobipy as grb
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats

# Q1

#### We have four variables, each of which represents the amount of production in Taipei, Taichung, Tainan and Taitung

x0: the amount of production in Taipei

x1: the amount of production in Taichung

x2: the amount of production in Tainan

x3: the amount of production in Taitung

In [ ]:
# Declare and initialize model
mLouisa = grb.Model('Louisa')

#I: The number of airfare classess
I=4

places={}
for i in range(I):
  places[i]=mLouisa.addVar(vtype=grb.GRB.CONTINUOUS, name='x'+str(i))

places

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>}

The constraints are:

1. **Total amount of production is up to 2000*0.6** (because "Regional demands shown in the table below total 2,000 pieces per week, but Louisa can produce only 60% of that number.")

2. According to "fulfills between 50 and 70% of each region’s demand.", **we can know that the amount of production in Taipei should be between 620 * 0.5 to 620 * 0.7 and so do other city.**

3. Those amount of production can't be negative, so they should be bigger than or equal to zero.

In [ ]:
mLouisa.addConstr(places[0]+places[1]+places[2]+places[3] <= 2000*0.6) #1
mLouisa.addConstr(places[0] <= (620*0.7)) #2
mLouisa.addConstr(places[0] >= (620*0.5))
mLouisa.addConstr(places[1] <= (490*0.7))
mLouisa.addConstr(places[1] >= (490*0.5))
mLouisa.addConstr(places[2] <= (510*0.7))
mLouisa.addConstr(places[2] >= (510*0.5))
mLouisa.addConstr(places[3] <= (380*0.7))
mLouisa.addConstr(places[3] >= (380*0.5))
mLouisa.addConstr(places[0] >= 0) #3
mLouisa.addConstr(places[1] >= 0)
mLouisa.addConstr(places[2] >= 0)
mLouisa.addConstr(places[3] >= 0)

<gurobi.Constr *Awaiting Model Update*>

The Obejective is:

**Louisa wants to find a profit-maximization plan**

Therefore, we sum over (the profit * the amount of production in every city), and we want to maximize that.

In [ ]:
objectiveLouisa = 1.60*places[0]+1.40*places[1]+1.90*places[2]+1.20*places[3]

mLouisa.setObjective(objectiveLouisa,grb.GRB.MAXIMIZE)
mLouisa.update()
mLouisa.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 4 columns and 16 nonzeros
Model fingerprint: 0x660b746c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 12 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 4 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9315000e+03   1.225000e+01   0.000000e+00      0s
       1    1.9021000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.902100000e+03


Some information about Louisa model

In [ ]:
print(mLouisa.Status == grb.GRB.OPTIMAL)
print(mLouisa.display())

True
Maximize
  1.6 x0 + 1.4 x1 + 1.9 x2 + 1.2 x3
Subject To
  R0: x0 + x1 + x2 + x3 <= 1200
  R1: x0 <= 434
  R2: x0 >= 310
  R3: x1 <= 343
  R4: x1 >= 245
  R5: x2 <= 357
  R6: x2 >= 255
  R7: x3 <= 266
  R8: x3 >= 190
  R9: x0 >= 0
  R10: x1 >= 0
  R11: x2 >= 0
  R12: x3 >= 0
None


Variables in Louisa model

In [ ]:
for v in mLouisa.getVars():
  print(v.VarName, v.X)

x0 408.0
x1 245.0
x2 357.0
x3 190.0


In [ ]:
optobj=mLouisa.getObjective()
print(optobj.getValue())

1902.1


# ANS1:
Our maxmized profit under those constraints is 1902.1.
To achieve that:
1. In Taipei, Louisa has to produce 408 muffins.
2. In Taichung, Louisa has to produce 245 muffins.
3. In Tainan, Louisa has to produce 357 muffins.
4. In Taitung, Louisa has to produce 190 muffins.

# Q2

We have 4 variables, each of which represents different ingredients(Oats, Corns, Alfalfa, Peanut Hulls)

x0: the amount of oat in a ton of feed.

x1: the amount of corn in a ton of feed.

x2: the amount of alfalfa in a ton of feed.

x3: the amount of peanut hulls in a ton of feed.

In [ ]:
mCattle = grb.Model('Cattle')

I=4

variables={}
for i in range(I):
  variables[i]=mCattle.addVar(vtype=grb.GRB.CONTINUOUS, name='x'+str(i))

variables

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>}

The constraints are:
1. "at least 60% of the daily allowance for protein and fiber."

2. "not allowing exceeding 60% of the fat allowance."

3. all the amount of ingredients can't be negative.

In [ ]:
mCattle.addConstr((0.6*variables[0]) + (0.8*variables[1]) + (0.55*variables[2]) + (0.4*variables[3]) >= 0.6)
mCattle.addConstr(((0.5*variables[0]) + (0.7*variables[1]) + (0.4*variables[2]) + (1*variables[3])) <= 0.6)
mCattle.addConstr(((0.9*variables[0]) + (0.3*variables[1]) + (0.6*variables[2]) + (0.8*variables[3])) >= 0.6)
mCattle.addConstr(variables[0]+variables[1]+variables[2]+variables[3] == 1)
mCattle.addConstr(variables[0] >= 0)
mCattle.addConstr(variables[1] >= 0)
mCattle.addConstr(variables[2] >= 0)
mCattle.addConstr(variables[3] >= 0)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
objCattle = 200*variables[0] + 150*variables[1] + 100*variables[2] + 75*variables[3]

mCattle.setObjective(objCattle,grb.GRB.MINIMIZE)
mCattle.update()
mCattle.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 4 columns and 20 nonzeros
Model fingerprint: 0x18ab4229
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [8e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 4 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+01   9.984000e-01   0.000000e+00      0s
       7    1.2500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.250000000e+02


In [ ]:
print(mCattle.Status == grb.GRB.OPTIMAL)
print(mCattle.display())

True
Minimize
  200.0 x0 + 150.0 x1 + 100.0 x2 + 75.0 x3
Subject To
  R0: 0.6 x0 + 0.8 x1 + 0.55 x2 + 0.4 x3 >= 0.6
  R1: 0.5 x0 + 0.7 x1 + 0.4 x2 + x3 <= 0.6
  R2: 0.9 x0 + 0.3 x1 + 0.6 x2 + 0.8 x3 >= 0.6
  R3: x0 + x1 + x2 + x3 = 1
  R4: x0 >= 0
  R5: x1 >= 0
  R6: x2 >= 0
  R7: x3 >= 0
None


In [ ]:
for v in mCattle.getVars():
  print(v.VarName, v.X)

x0 0.15714285714285703
x1 0.27142857142857124
x2 0.40000000000000024
x3 0.17142857142857149


In [ ]:
optobj=mCattle.getObjective()
print(optobj.getValue())

124.99999999999997


# ANS2

Our minimial cost under those constraints is 124.999999999....
To achieve that:
1. We have to add 0.157.. tons of Oats into the feed.
2. We have to add 0.271.. tons of Corns into the feed.
3. We have to add 0.400.. tons of Alfalfa into the feed.
4. We have to add 0.171.. tons of Peanuts into the feed.

# Q3

#### In this model, there are 4 variables , each of which represents 4 different types of gasoline.

x0: gasoline with 99 and 210

x1: gasoline with 70 and 335

x2: gasoline with 78 and 280

x3: gasoline with 91 and 265

In [ ]:
mGas = grb.Model('Gasoline')

I=4

gas={}
for i in range(I):
  gas[i]=mGas.addVar(vtype=grb.GRB.CONTINUOUS, name='x'+str(i))

gas

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>}

#### From the instruction, we can know that:
constraints:

1. "blend with a first quality index between 85 and 90."

2. "and a second quality index between 270 and 280."

3. the sum over 4 types of gasoline should equal to 1(beacuse we want to know the proportion in a blend per barrel).

4. all the amount of gasoline can't be negative.


In [ ]:
mGas.addConstr((99*gas[0] + 70*gas[1] + 78*gas[2] + 91*gas[3]) >= (85*(gas[0]+gas[1]+gas[2]+gas[3])))
mGas.addConstr((99*gas[0] + 70*gas[1] + 78*gas[2] + 91*gas[3]) <= (90*(gas[0]+gas[1]+gas[2]+gas[3])))
mGas.addConstr((210*gas[0] + 335*gas[1] + 280*gas[2] + 265*gas[3]) >= (270*(gas[0]+gas[1]+gas[2]+gas[3])))
mGas.addConstr((210*gas[0] + 335*gas[1] + 280*gas[2] + 265*gas[3]) <= (280*(gas[0]+gas[1]+gas[2]+gas[3])))
mGas.addConstr((gas[0]+gas[1]+gas[2]+gas[3]) == 1)
mGas.addConstr(gas[0] >= 0)
mGas.addConstr(gas[1] >= 0)
mGas.addConstr(gas[2] >= 0)
mGas.addConstr(gas[3] >= 0)

<gurobi.Constr *Awaiting Model Update*>

The objective is:

"we would like to choose a cost-minimizing blend."

so we multiply each price of gasoline with its proportion and sum over them to get the total cost.

and we want to minimize this cost.

In [ ]:
objGas = 48*gas[0] + 43*gas[1] + 58*gas[2] + 46*gas[3]

mGas.setObjective(objGas,grb.GRB.MINIMIZE)
mGas.update()
mGas.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 4 columns and 23 nonzeros
Model fingerprint: 0x06c223d4
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 4 columns, 19 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3000000e+01   1.062500e+01   0.000000e+00      0s
       2    4.5294118e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.529411765e+01


In [ ]:
print(mGas.Status == grb.GRB.OPTIMAL)
print(mGas.display())

True
Minimize
  48.0 x0 + 43.0 x1 + 58.0 x2 + 46.0 x3
Subject To
  R0: 14.0 x0 + -15.0 x1 + -7.0 x2 + 6.0 x3 >= 0
  R1: 9.0 x0 + -20.0 x1 + -12.0 x2 + x3 <= 0
  R2: -60.0 x0 + 65.0 x1 + 10.0 x2 + -5.0 x3 >= 0
  R3: -70.0 x0 + 55.0 x1 + -15.0 x3 <= 0
  R4: x0 + x1 + x2 + x3 = 1
  R5: x0 >= 0
  R6: x1 >= 0
  R7: x2 >= 0
  R8: x3 >= 0
None


In [ ]:
for v in mGas.getVars():
  print(v.VarName, v.X)

x0 0.17647058823529413
x1 0.35294117647058826
x2 0.0
x3 0.47058823529411753


In [ ]:
optobj=mGas.getObjective()
print(optobj.getValue())

45.29411764705882


#ANS3

Our minimial cost per barrel under those constraints is 45.29411764705882.
To achieve that:
1. gasoline with quality being 99 and 210 should occupy 0.176... of a blend.
2. gasoline with quality being 70 and 335 should occupy 0.352... of a blend.

3. gasoline with quality being 78 and 280 should occupy 0 of a blend.
(we don't have to add this gasoline into blend according to the model.)
4. gasoline with quality being 91 and 265 should occupy 0.47... of a blend.